In [1]:
#################################STUDY POINTS #######################
#1. to check if any null value in whole df
# df.isnull().any().any()
#2. To get all rows which having alteast one nan value
# nan_values=testdf[df.isnull().T.any()]
# 3. Create new column/columns based on single or multiple conditions 
# np.where(condition, value if condition is true, value if condition is false)
# eqdf['SDirection']=np.where(eqdf['Close']>eqdf['Pclose'],1,-1)

# # create a list of our conditions
# conditions = [
#     (df['likes_count'] <= 2),
#     (df['likes_count'] > 2) & (df['likes_count'] <= 9),
#     (df['likes_count'] > 9) & (df['likes_count'] <= 15),
#     (df['likes_count'] > 15)
#     ]

# # create a list of the values we want to assign for each condition
# values = ['tier_4', 'tier_3', 'tier_2', 'tier_1']

# # create a new column and use np.select to assign values to it using our lists as arguments
# df['tier'] = np.select(conditions, values)

In [3]:
import pandas as pd
import numpy as np
from pymongo import MongoClient
import matplotlib.pyplot as plt
from jupyter_dash import JupyterDash
import plotly.graph_objects as go
import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.figure_factory as ff

In [4]:
mongo_client=MongoClient()
db=mongo_client['OPTIONCHAIN']

In [5]:
# ##############This take time to get data from DB ###########
# ###############Run it again if you want to fetch fresh data from db######
# dforignal=pd.DataFrame(list(db.NIFTY.find()))
# dforignal.drop("_id",axis=1,inplace=True)

In [6]:
dfc=dforignal.copy()
dfc['datetime']=pd.to_datetime(dfc['datetime'])
dfc['expiry_date']=pd.to_datetime(dfc['expiry_date'])
dfc.set_index('datetime',inplace=True)

In [7]:
def atm_data(day_date,expiry):
    day_df=dfc.loc[(dfc.index==day_date+' 09:20:00')& (dfc['expiry_date']==expiry)]
    if not day_df.empty:
        ####ATM Call and Put Strikes
        atmcallstrike=float(day_df.iloc[(day_df['CallDelta']-50).abs().argsort()[:1]]['Strikes'])
        atmputstrike=float(day_df.iloc[(day_df['PutDelta']+50).abs().argsort()[:1]]['Strikes'])

        day_atm_call_df=dfc.loc[(dfc.index<=day_date+' 15:20:00')& (dfc['expiry_date']==expiry)&(dfc['Strikes']==atmcallstrike)][['CallLTP','Strikes','expiry_date']]
        day_atm_put_df=dfc.loc[(dfc.index<=day_date+' 15:20:00')& (dfc['expiry_date']==expiry)&(dfc['Strikes']==atmputstrike)][['PutLTP','Strikes','expiry_date']]
        
        day_atm_call_df.replace(to_replace=0,method='ffill',inplace=True)
        day_atm_put_df.replace(to_replace=0,method='ffill',inplace=True)
        
        return day_atm_call_df,day_atm_put_df

In [18]:
def mv(sma,calldf,putdf):
    for sma in sma:
        calldf[f'{sma}sma']=calldf['CallLTP'].rolling(sma).mean().bfill()
        putdf[f'{sma}sma']=putdf['PutLTP'].rolling(sma).mean().bfill()
    
    return calldf,putdf

In [22]:
atm_call,atm_put=atm_data('2019-1-30','2019-1-31')

In [20]:
# sma=[21,11]
# atm_call,atm_put=mv(sma,atm_call,atm_put)

In [52]:
def stratergy_tester(startdate,enddate):
    atm_call,atm_put=atm_data('2019-1-30','2019-1-31')
    ##Long Sma before short one 
    sma=[100,26]
    atm_call,atm_put=mv(sma,atm_call,atm_put)
    call_conditions=[(atm_call['CallLTP']<atm_call[f'{sma[0]}sma'])&(atm_call['CallLTP'].shift(1).bfill()>atm_call[f'{sma[0]}sma']),
                (atm_call['CallLTP']<atm_call[f'{sma[0]}sma'])&(atm_call['CallLTP']>atm_call[f'{sma[1]}sma'])&(atm_call['CallLTP'].shift(1).bfill()<atm_call[f'{sma[1]}sma'])
               ]
    
    values=[1,0]
    
    atm_call['signal']=np.select(call_conditions,values)
    return atm_call

In [53]:
atm_call=stratergy_tester('2019-1-1','2019-1-31')

In [55]:
atm_call.to_csv('datacheck.csv')

In [46]:
### This whole section to plot charts in Dash
def genrate_graphs():
    app = JupyterDash(__name__)
    app.layout = html.Div(children=[
            html.H1(children='Hello Dash'),

            html.Div(children='''
                Dash: A web application framework for your data.
            '''),
            dcc.Checklist(
                id='toggle-rangeslider',
                options=[{'label': 'Include Rangeslider', 
                          'value': 'slider'}],
                value=['slider']
            ),
            dcc.Graph(
                id='graph',
                # figure=fig
            )
        ])
    @app.callback(Output("graph", "figure"), [Input("toggle-rangeslider", "value")])
    def show_chart(value):
        fig=make_subplots(rows=2,cols=1,column_widths=[2.0],
                                  row_heights=[2.0,1.0],shared_xaxes=True,vertical_spacing=0.01,
                                  specs=[
                                      [{"type":"xy","secondary_y": True}],#None]])
                                      [{"type":"xy"}],#None]]
                                        ]
                                 )
 
        fig.add_trace(go.Scatter(x=atm_call.index,y=atm_call['CallLTP'], name="ATMCall",mode='lines'),row=1,col=1,secondary_y=False)
        #fig.add_trace(go.Scatter(x=atm_call.index,y=atm_call[f'{sma}sma'], name=f"{sma}sma",mode='lines'),row=1,col=1,secondary_y=False)
        fig.add_trace(go.Scatter(x=atm_call.index,y=atm_call['signal'], name="signal",mode='lines'),row=1,col=1,secondary_y=True)
        # fig.add_trace(go.Scatter(x=atm_put.index,y=atm_put['PutLTP'], name="ATMPut",mode='lines'),row=2,col=1)
        #fig.add_trace(go.Scatter(x=atm_put.index,y=atm_put[f'{sma}sma'], name=f"{sma}sma",mode='lines'),row=2,col=1)
        
        # fig.add_trace(go.Scatter(x=itm_options.index,y=itm_options['CallLTP'], name="ITMCall",mode='lines'),row=2,col=1)
        # fig.add_trace(go.Scatter(x=itm_options.index,y=itm_options['50macall'], name="ITMCall50ma",mode='lines'),row=2,col=1)
        # fig.add_trace(go.Scatter(x=itm_options.index,y=itm_options['PutLTP'], name="ITMPut",mode='lines'),row=2,col=1)
        # fig.add_trace(go.Scatter(x=itm_options.index,y=itm_options['50maput'], name="ITMPut50ma",mode='lines'),row=2,col=1)
        # fig.add_trace(go.Scatter(x=atm_options.index,y=atm_options['PutLTP'], name="Put",mode='lines'),row=2,col=1)
        fig.update_xaxes(row=1,col=1,
                         # rangeselector=dict(buttons=list(
                         #     [dict(count=1, label="1m", step="month", stepmode="backward"),
                         #      dict(count=6, label="6m", step="month", stepmode="backward"),
                         #      dict(count=1, label="YTD", step="year", stepmode="todate"),
                         #      dict(count=1, label="1y", step="year", stepmode="backward"),
                         #      dict(step="all")
                         #     ])
                         #                   ),
                         rangebreaks=[
                             dict(bounds=["sat", "mon"]),#hide weekends
                             dict(bounds=[15.5, 9.25], pattern="hour")
                         ]
                        )
                # rangebreaks=[
            #     dict(bounds=[15, 9], pattern="hour"), #hide hours outside of 9am-5pm
            # ]
        fig.update_layout(autosize=False,width=1200,height=800,
                          margin=dict(l=50,r=50,b=100,t=100,pad=4),
                          paper_bgcolor="LightSteelBlue",)
        return fig
    app.run_server(mode='jupyterlab',port = 8098)

In [54]:
genrate_graphs()